In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from datetime import date, datetime
from lib.predict import (
    generate_sequences, SequenceDataset, LSTMForecaster, LSTM, 
    make_predictions_from_dataloader
)
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

True
True


In [3]:
spydata = pd.read_csv("spydata.csv")
oil = spydata[["Date", "Close", "Volume"]]
# oil.loc[:,"Date"] = pd.to_datetime(oil.Date)
oil = oil.set_index('Date').interpolate()
# print(oil.isna().sum())
df = oil.copy().dropna(axis=0)

In [4]:
scalers = {}
for x in df.columns:
  scalers[x] = StandardScaler().fit(df[x].values.reshape(-1, 1))

norm_df = df.copy()
for i, key in enumerate(scalers.keys()):
  norm = scalers[key].transform(norm_df.iloc[:, i].values.reshape(-1, 1))
  norm_df.iloc[:, i] = norm

In [5]:
BATCH_SIZE = 16
nhid = 5
sequence_len = 180
n_dnn_layers = 5
ninp = 2
nout = 2
split = 0.8
n_epochs = 40

sequences = generate_sequences(norm_df[["Close", "Volume"]], sequence_len, nout, ["Close", "Volume"])
dataset = SequenceDataset(sequences)

train_len = int(len(dataset)*split)
lens = [train_len, len(dataset)-train_len]
train_ds, test_ds = random_split(dataset, lens)


In [6]:
trainloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
testloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [7]:
def plot_losses(tr, va):
  import matplotlib.pyplot as plt
  fig, ax = plt.subplots()
  ax.plot(tr, label='train')
  ax.plot(va, label='validation')
  plt.show()

In [8]:
class LSTMForecaster(nn.Module):

  def __init__(self, n_features, n_hidden, n_outputs, sequence_len, n_lstm_layers=1, n_deep_layers=10, use_cuda=False, dropout=0.2):
    '''
    n_features: number of input features (1 for univariate forecasting)
    n_hidden: number of neurons in each hidden layer
    n_outputs: number of outputs to predict for each training example
    n_deep_layers: number of hidden dense layers after the lstm layer
    sequence_len: number of steps to look back at for prediction
    dropout: float (0 < dropout < 1) dropout ratio between dense layers
    '''
    super().__init__()

    self.n_lstm_layers = n_lstm_layers
    self.nhid = n_hidden
    self.use_cuda = use_cuda # set option for device selection

    # LSTM Layer
    self.lstm = nn.LSTM(n_features,
                        n_hidden,
                        num_layers=n_lstm_layers,
                        batch_first=True) # As we have transformed our data in this way
    
    # first dense after lstm
    self.fc1 = nn.Linear(n_hidden * sequence_len, n_hidden) 
    # Dropout layer 
    self.dropout = nn.Dropout(p=dropout)

    # Create fully connected layers (n_hidden x n_deep_layers)
    dnn_layers = []
    for i in range(n_deep_layers):
      # Last layer (n_hidden x n_outputs)
      if i == n_deep_layers - 1:
        dnn_layers.append(nn.ReLU())
        dnn_layers.append(nn.Linear(self.nhid, n_outputs))
      # All other layers (n_hidden x n_hidden) with dropout option
      else:
        dnn_layers.append(nn.ReLU())
        dnn_layers.append(nn.Linear(self.nhid, self.nhid))
        if dropout:
          dnn_layers.append(nn.Dropout(p=dropout))
    # compile DNN layers
    self.dnn = nn.Sequential(*dnn_layers)

  def forward(self, x):

    # Initialize hidden state
    hidden_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)
    cell_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)

    # move hidden state to device
    if self.use_cuda:
      hidden_state = hidden_state.to(device)
      cell_state = cell_state.to(device)
        
    self.hidden = (hidden_state, cell_state)

    # Forward Pass
    x, h = self.lstm(x, self.hidden) # LSTM
    x = self.dropout(x.contiguous().view(x.shape[0], -1)) # Flatten lstm out 
    x = self.fc1(x) # First Dense
    return self.dnn(x) # Pass forward through fully connected DNN.



In [9]:
USE_CUDA = torch.cuda.is_available()
device = 'cuda' if USE_CUDA else 'cpu'
lr = 0.01
model =  LSTMForecaster(
            n_features=ninp, 
            n_hidden=nhid, 
            n_outputs=nout, 
            sequence_len=sequence_len, 
            n_lstm_layers=1,
            n_deep_layers=n_dnn_layers, 
            use_cuda=USE_CUDA,
            dropout=0.2
        ).to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [10]:
t_losses, v_losses = [], []
for epoch in range(n_epochs):
  train_loss, valid_loss = 0.0, 0.0


  # train step
  model.train()
  for x, y in trainloader:
    optimizer.zero_grad()
    x = x.to(device)
    y = y.squeeze().to(device)   

    # print(len(x))
    # print(len(y))
    
    # (2in,1out) x-shape = torch.Size([20, 180, 2]), y-shape = torch.Size([20])
    # (2in,2out) x-shape = torch.Size([16, 180, 2]), y-shape = torch.Size([16, 2, 2]), preds-shape = torch.Size([16, 2])
    # Forward Pass
    preds = model(x).squeeze()    
    print(f"==========^^^^^^^^")
    print(f"x-shape = {x.shape}, y-shape = {y.shape}, preds-shape = {preds.shape}")

    loss = criterion(preds, y)
    print(f"******")
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
  epoch_loss = train_loss / len(trainloader)
  t_losses.append(epoch_loss)
  
  # validation step
  model.eval()
  for x, y in testloader:
    with torch.no_grad():
      x, y = x.to(device), y.squeeze().to(device)
      preds = model(x).squeeze()
      error = criterion(preds, y)
    valid_loss += error.item()
  valid_loss = valid_loss / len(testloader)
  v_losses.append(valid_loss)
      
  print(f'{epoch} - train: {epoch_loss}, valid: {valid_loss}')

plot_losses(t_losses, v_losses)

==========^^^^^^^^
x-shape = torch.Size([16, 180, 2]), y-shape = torch.Size([16, 2, 2]), preds-shape = torch.Size([16, 2])


/Users/meetri/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16, 2, 2])) that is different to the input size (torch.Size([16, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (16) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
class Forecaster:

  def __init__(self, model, data, target, tw):
    self.model = model
    self.data = data
    self.tw = tw
    self.target = target

  def plot_forecast(self, history):
    fig = go.Figure()
    # Add traces
    fig.add_trace(go.Scatter(x=history.index, y=history.actual,
                        mode='lines',
                        name='actual'))
    fig.add_trace(go.Scatter(x=history.index, y=history.forecast,
                        mode='lines',
                        name='forecast'))
    fig.update_layout(
    autosize=False,
    width=2400,
    height=800,)
    fig.show()
  
  def one_step_forecast(self, history):
      '''
      history: a sequence of values representing the latest values of the time 
      series, requirement -> len(history.shape) == 2

      outputs a single value which is the prediction of the next value in the
      sequence.
      '''
      self.model.cpu()
      self.model.eval()
      with torch.no_grad():
        pre = torch.Tensor(history).unsqueeze(0)
        pred = self.model(pre)
      return pred.detach().numpy().reshape(-1)

  def n_step_forecast(self, n: int, forecast_from: int=None, plot=False):
      '''
      n: integer defining how many steps to forecast
      forecast_from: integer defining which index to forecast from. None if
      you want to forecast from the end.
      plot: True if you want to output a plot of the forecast, False if not.
      '''
      history = self.data[self.target] # .to_frame()
    
      # print(history)
      # Create initial sequence input based on where in the series to forecast 
      # from.
      if forecast_from:
        pre = list(history[forecast_from - self.tw : forecast_from][self.target].values)
      else:
        pre = history[self.target].values[-self.tw:].tolist()
        
      # Call one_step_forecast n times and append prediction to history
      for i, step in enumerate(range(n)):
        pre_ = np.array(pre[-self.tw:])  # .reshape(-1, 1)
        forecast = self.one_step_forecast(pre_).squeeze()
        # pre.append(forecast)

      res = history.copy()
      ls = [np.nan for i in range(len(history))]

      # Note: I have not handled the edge case where the start index + n crosses
      # the end of the dataset
      if forecast_from:
        ls[forecast_from : forecast_from + n] = list(np.array(pre[-n:]))
        res['forecast'] = ls
        res.columns = ['actual', 'forecast']
      else:
        fc = ls + list(np.array(pre[-n:]))
        ls = ls + [np.nan for i in range(len(pre[-n:]))]
        ls[:len(history)] = history[self.target].values
        res = pd.DataFrame([ls, fc], index=['actual', 'forecast']).T

      if plot:
        self.plot_forecast(res)
      return res

In [ ]:
import plotly.graph_objects as go
fc = Forecaster(model, norm_df, ["Close", "Volume"], 180)

In [ ]:
history = fc.n_step_forecast(200, plot=True)

In [ ]:
import matplotlib.pyplot as plt
f = history[["forecast"]].dropna()
a = history[["actual"]].dropna()

x = list(zip(*a["actual"].values))
x2 = list(zip(*f["forecast"].values))
xb = pd.DataFrame(np.empty_like(a.actual))

# xb.loc(len(x2[0]), "forcast") = x2
xb.iloc[-180:, 0] = x2[0]

plt.plot(x[0]), plt.plot(xb[0])